In [1]:
import pandas as pd

In [2]:
df = pd.read_excel('Новосибирск-5.xls', index_col=0, usecols='A:I')
df = df[(df['урожай'] != 0) | (df['культура'] == 'пар')]
df.head()


,Севооборот,минус 3 года,минус 2 года,минус 1 год,культура,год,урожай,примечание
№,,,,,,,,
1,0,пшеница,пшеница,пшеница,пшеница,1999,11.1,предшественники фиктивные
2,0,пшеница,пшеница,пшеница,пшеница,2000,31.7,бессменная пшеница
3,0,пшеница,пшеница,пшеница,пшеница,2001,25.3,NaN
4,0,пшеница,пшеница,пшеница,пшеница,2002,13.6,NaN
5,0,пшеница,пшеница,пшеница,пшеница,2003,8.2,NaN


## Промежуточная матрица с известными значениями

In [3]:
def find_avg(df, forbear, crop):
    if crop == 'пар':
        return 0, 'получено экспереметально'
    
    # Фильтрация строк по значениям в столбцах 'минус 1 год' и 'культура'
    filtered_df = df[(df['минус 1 год'] == forbear) & (df['культура'] == crop)]
    
    # Если есть совпадения, возвращаем значение из столбца 'avg'
    if not filtered_df.empty:
        return filtered_df['урожай'].mean(), 'получено экспереметально'
    return None, None
    


crops = {'предшественник':[], 'культура':[], 'avg':[], 'примечание -1 год':[]}
unique_crop = df['культура'].unique()
for forbear in unique_crop:
    for crop in unique_crop:
        crops['предшественник'].append(forbear)
        crops['культура'].append(crop)
        avg, remark = find_avg(df, forbear, crop)
        crops['avg'].append(avg)
        crops['примечание -1 год'].append(remark)
tensor = pd.DataFrame(crops)
pd.set_option('display.max_rows', None)
tensor


,предшественник,культура,avg,примечание -1 год
0,пшеница,пшеница,17.599048,получено экспереметально
1,пшеница,овес,18.985714,получено экспереметально
2,пшеница,ячмень,20.750000,получено экспереметально
3,пшеница,пар,0.000000,получено экспереметально
4,пшеница,оз. рожь,NaN,None
5,пшеница,донник(вика)клевер,NaN,None
6,пшеница,вика-овес,NaN,None
7,пшеница,рапс,NaN,None
8,пшеница,клевер,NaN,None
9,пшеница,вика-овес (з/м),NaN,None


## Заполняем пустые ячейки, умножая среднюю урожайность культуры на среднее влияние, которое оказывает предшественник

In [4]:
import numpy as np

def calculate_avg(df, forbear, crop):
    df_forbear = df[(df['минус 1 год'] == forbear) & (df['культура'] != 'пар')]
    unique_crop = df_forbear['культура'].unique()
    if unique_crop.size == 0:
        return None, None
    odds = [df_forbear[df_forbear['культура'] == crop]['урожай'].mean() 
            / df[df['культура'] == crop]['урожай'].mean() for crop in unique_crop]
    return sum(odds) / len(odds) * df[df['культура'] == crop]['урожай'].mean(), unique_crop


for index, row in tensor.iterrows():
    forbear, crop, avg = row['предшественник'], row['культура'], row['avg']
    
    if pd.isna(avg):
        calculate_avg_result, source = calculate_avg(df, forbear, crop)
        if calculate_avg_result is not None: 
            tensor.at[index, 'avg'] = calculate_avg_result
            tensor.at[index, 'примечание -1 год'] = f'получено из {", ".join(source)}'

     
tensor


,предшественник,культура,avg,примечание -1 год
0,пшеница,пшеница,17.599048,получено экспереметально
1,пшеница,овес,18.985714,получено экспереметально
2,пшеница,ячмень,20.750000,получено экспереметально
3,пшеница,пар,0.000000,получено экспереметально
4,пшеница,оз. рожь,27.769808,"получено из пшеница, овес, ячмень"
5,пшеница,донник(вика)клевер,38.244246,"получено из пшеница, овес, ячмень"
6,пшеница,вика-овес,23.592412,"получено из пшеница, овес, ячмень"
7,пшеница,рапс,5.908361,"получено из пшеница, овес, ячмень"
8,пшеница,клевер,39.717313,"получено из пшеница, овес, ячмень"
9,пшеница,вика-овес (з/м),26.973991,"получено из пшеница, овес, ячмень"


## Исправляем значение, где предшественник донник(вика)клевер, потому что он эксперементально влиял только на оз. рожь, которая после него выращивалась ещё лишь с паром
correct_avg = урожайности после донник(вика)клевера[54] / урожайности после пара[34]  оз.ржи
            * уражайность другой культуры после пара

In [5]:
def correct_avg(row):
    if row['предшественник'] == 'донник(вика)клевер':
        corect_avg = tensor['avg'][54]/tensor['avg'][34] * tensor[(tensor['предшественник']=='пар') & (tensor['культура'] == row['культура'])]['avg'].values[0] 
        return corect_avg
    return row['avg']
def correct_remark(row):
    if row['предшественник'] == 'донник(вика)клевер':
        return 'получено из влияния донник(вика)клевера и пара на оз. рожь'
    return row['примечание -1 год']
tensor['avg'] = tensor.apply(correct_avg, axis=1)
tensor['примечание -1 год'] = tensor.apply(correct_remark, axis=1)
tensor

,предшественник,культура,avg,примечание -1 год
0,пшеница,пшеница,17.599048,получено экспереметально
1,пшеница,овес,18.985714,получено экспереметально
2,пшеница,ячмень,20.750000,получено экспереметально
3,пшеница,пар,0.000000,получено экспереметально
4,пшеница,оз. рожь,27.769808,"получено из пшеница, овес, ячмень"
5,пшеница,донник(вика)клевер,38.244246,"получено из пшеница, овес, ячмень"
6,пшеница,вика-овес,23.592412,"получено из пшеница, овес, ячмень"
7,пшеница,рапс,5.908361,"получено из пшеница, овес, ячмень"
8,пшеница,клевер,39.717313,"получено из пшеница, овес, ячмень"
9,пшеница,вика-овес (з/м),26.973991,"получено из пшеница, овес, ячмень"


## Добавляем коэфиценты

In [6]:
avg_par = tensor[tensor['предшественник'] == 'пар'].set_index('культура')['avg']
with np.errstate(divide='ignore', invalid='ignore'):    
    tensor['коэффициент -1 год относительно пара'] = tensor.apply(lambda row: row['avg'] / avg_par[row['культура']], axis=1)
tensor

,предшественник,культура,avg,примечание -1 год,коэффициент -1 год относительно пара
0,пшеница,пшеница,17.599048,получено экспереметально,0.550525
1,пшеница,овес,18.985714,получено экспереметально,0.712560
2,пшеница,ячмень,20.750000,получено экспереметально,0.712560
3,пшеница,пар,0.000000,получено экспереметально,NaN
4,пшеница,оз. рожь,27.769808,"получено из пшеница, овес, ячмень",0.783050
5,пшеница,донник(вика)клевер,38.244246,"получено из пшеница, овес, ячмень",0.682186
6,пшеница,вика-овес,23.592412,"получено из пшеница, овес, ячмень",0.682186
7,пшеница,рапс,5.908361,"получено из пшеница, овес, ячмень",0.682186
8,пшеница,клевер,39.717313,"получено из пшеница, овес, ячмень",0.682186
9,пшеница,вика-овес (з/м),26.973991,"получено из пшеница, овес, ячмень",0.682186


## Добавляем коэфиценты за 2 год

In [10]:
# словарь для быстрого доступа к коэффициентам
coeff_1_year = dict(zip(zip(tensor['предшественник'], tensor['культура']), tensor['коэффициент -1 год относительно пара']))

# Добавление новых столбцов в tensor
tensor['коэффициент -2 года относительно пара'] = None
tensor['примечание -2 года'] = None

# Функция для расчета коэффициента за 2 года
def calculate_2_year_odds(row):
    harvest2 = []
    for i, df_row in df.iterrows():
        if (df_row['минус 2 года'], df_row['культура']) == (row['предшественник'], row['культура']):
            # Рассчитываем урожай без влияния культур за 1 год
            adjusted_crop = df_row['урожай'] / coeff_1_year[(df_row['минус 1 год'], df_row['культура'])]
            harvest2.append(adjusted_crop)
    avg_crop_for_par = tensor[(tensor['предшественник'] == 'пар') & (tensor['культура'] == row['культура'])]['avg'].values[0]
    row['коэффициент -2 года относительно пара'] = sum(harvest2)/len(harvest2) / avg_crop_for_par
    row['примечание -2 года'] = 'получено экспереметально'
    return row
    
def find_odds2(row):
    if row['культура'] == 'пар':
        row['коэффициент -2 года относительно пара'], row['примечание -2 года'] = np.NaN, 'получено экспереметально'
        return row
    filtered_df = df[(df['минус 2 года'] == row['предшественник']) & (df['культура'] == row['культура'])]
    if not filtered_df.empty:
        return calculate_2_year_odds(row)
    else:
        df_forbear = df[(df['минус 2 года'] == row['предшественник']) & (df['культура'] != 'пар')]
        unique_crop = df_forbear['культура'].unique()
        if unique_crop.size == 0:
            return None, None
        odds = []
        for crop in unique_crop:
            
            # делим урожайность crop на второй год после row['предшественник'] на коэффициент культуры предшественика в прошлом году
            series_forbear_with_crop = df_forbear[df_forbear['культура'] == crop].set_index('минус 1 год')['урожай']/tensor[(tensor['культура'] == crop)].set_index('предшественник')['коэффициент -1 год относительно пара']
            
            #урожайность crop в целом после пара
            series_crop = tensor[(tensor['культура'] == crop)&(tensor['предшественник'] == 'пар')]['avg']
            
            odds.append(series_forbear_with_crop.mean()/series_crop.mean())
        row['коэффициент -2 года относительно пара'] = sum(odds) / len(odds)
        row['примечание -2 года'] = f'получено из {", ".join(unique_crop)}'
        return row


tensor = tensor.apply(find_odds2, axis=1)
tensor


,предшественник,культура,avg,примечание -1 год,коэффициент -1 год относительно пара,коэффициент -2 года относительно пара,примечание -2 года
0,пшеница,пшеница,17.599048,получено экспереметально,0.550525,0.954381,получено экспереметально
1,пшеница,овес,18.985714,получено экспереметально,0.712560,0.990260,"получено из пшеница, ячмень, донник(вика)клеве..."
2,пшеница,ячмень,20.750000,получено экспереметально,0.712560,0.977442,получено экспереметально
3,пшеница,пар,0.000000,получено экспереметально,NaN,NaN,получено экспереметально
4,пшеница,оз. рожь,27.769808,"получено из пшеница, овес, ячмень",0.783050,0.990260,"получено из пшеница, ячмень, донник(вика)клеве..."
5,пшеница,донник(вика)клевер,38.244246,"получено из пшеница, овес, ячмень",0.682186,1.000000,получено экспереметально
6,пшеница,вика-овес,23.592412,"получено из пшеница, овес, ячмень",0.682186,1.000000,получено экспереметально
7,пшеница,рапс,5.908361,"получено из пшеница, овес, ячмень",0.682186,1.000000,получено экспереметально
8,пшеница,клевер,39.717313,"получено из пшеница, овес, ячмень",0.682186,1.000000,получено экспереметально
9,пшеница,вика-овес (з/м),26.973991,"получено из пшеница, овес, ячмень",0.682186,1.000000,получено экспереметально


## Добавляем коэфиценты за 3 год

In [20]:
# словарь для быстрого доступа к коэффициентам
coeff_2_year = dict(zip(zip(tensor['предшественник'], tensor['культура']), tensor['коэффициент -2 года относительно пара']))

# Добавление новых столбцов в tensor
tensor['коэффициент -3 года относительно пара'] = None
tensor['примечание -3 года'] = None

# Функция для расчета коэффициента за 3 года
def calculate_3_year_odds(row):
    harvest3 = []
    for i, df_row in df.iterrows():
        if (df_row['минус 3 года'], df_row['культура']) == (row['предшественник'], row['культура']):
            # Рассчитываем урожай без влияния культур за 1 и 2 года
            adjusted_crop = df_row['урожай'] / coeff_1_year[(df_row['минус 1 год'], df_row['культура'])] / coeff_2_year[(df_row['минус 2 года'], df_row['культура'])]
            harvest3.append(adjusted_crop)
    avg_crop_for_par = tensor[(tensor['предшественник'] == 'пар') & (tensor['культура'] == row['культура'])]['avg'].values[0]
    row['коэффициент -3 года относительно пара'] = sum(harvest3)/len(harvest3) / avg_crop_for_par
    row['примечание -3 года'] = 'получено экспереметально'
    return row


def find_odds3(row):
    if row['культура'] == 'пар':
        row['коэффициент -3 года относительно пара'], row['примечание -3 года'] = np.NaN, 'получено экспереметально'
        return row
    
    filtered_df = df[(df['минус 3 года'] == row['предшественник']) & (df['культура'] == row['культура'])]
    
    if not filtered_df.empty:
        return calculate_3_year_odds(row)
    else:
        df_forbear = df[(df['минус 3 года'] == row['предшественник']) & (df['культура'] != 'пар')]
        unique_crop = df_forbear['культура'].unique()
        if unique_crop.size == 0:
            return None, None
        odds = []
        for crop in unique_crop:
            list_odds = []
            #урожайность crop в целом после пара
            series_crop = tensor[(tensor['культура'] == crop)&(tensor['предшественник'] == 'пар')]['avg']
            for i, df_row in df.iterrows():
                if (df_row['минус 3 года'], df_row['культура']) == (row['предшественник'], crop):
                    adjusted_crop = df_row['урожай'] / coeff_1_year[(df_row['минус 1 год'], df_row['культура'])] / coeff_2_year[(df_row['минус 2 года'], df_row['культура'])]
                    list_odds.append(adjusted_crop)            
            odds.append(sum(list_odds)/len(list_odds)/series_crop.mean())
        
        row['коэффициент -3 года относительно пара'] = sum(odds) / len(odds)
        row['примечание -3 года'] = f'получено из {", ".join(unique_crop)}'
        return row

tensor = tensor.apply(find_odds3, axis=1)
tensor

,предшественник,культура,avg,коэффициент -1 год относительно пара,примечание -1 год,коэффициент -2 года относительно пара,примечание -2 года,коэффициент -3 года относительно пара,примечание -3 года
0,пшеница,пшеница,17.599048,0.550525,получено экспереметально,0.954381,получено экспереметально,0.981060,получено экспереметально
1,пшеница,овес,18.985714,0.712560,получено экспереметально,0.990260,"получено из пшеница, ячмень, донник(вика)клеве...",1.000000,получено экспереметально
2,пшеница,ячмень,20.750000,0.712560,получено экспереметально,0.977442,получено экспереметально,1.000000,получено экспереметально
3,пшеница,пар,0.000000,NaN,получено экспереметально,NaN,получено экспереметально,NaN,получено экспереметально
4,пшеница,оз. рожь,27.769808,0.783050,"получено из пшеница, овес, ячмень",0.990260,"получено из пшеница, ячмень, донник(вика)клеве...",1.000000,получено экспереметально
5,пшеница,донник(вика)клевер,38.244246,0.682186,"получено из пшеница, овес, ячмень",1.000000,получено экспереметально,0.997633,"получено из пшеница, овес, ячмень, оз. рожь, в..."
6,пшеница,вика-овес,23.592412,0.682186,"получено из пшеница, овес, ячмень",1.000000,получено экспереметально,1.000000,получено экспереметально
7,пшеница,рапс,5.908361,0.682186,"получено из пшеница, овес, ячмень",1.000000,получено экспереметально,1.000000,получено экспереметально
8,пшеница,клевер,39.717313,0.682186,"получено из пшеница, овес, ячмень",1.000000,получено экспереметально,1.000000,получено экспереметально
9,пшеница,вика-овес (з/м),26.973991,0.682186,"получено из пшеница, овес, ячмень",1.000000,получено экспереметально,1.000000,получено экспереметально


In [21]:
new_order = ['предшественник', 'культура', 'avg', 'коэффициент -1 год относительно пара', 'примечание -1 год', 'коэффициент -2 года относительно пара', 'примечание -2 года', 'коэффициент -3 года относительно пара', 'примечание -3 года']
tensor = tensor[new_order]
tensor

,предшественник,культура,avg,коэффициент -1 год относительно пара,примечание -1 год,коэффициент -2 года относительно пара,примечание -2 года,коэффициент -3 года относительно пара,примечание -3 года
0,пшеница,пшеница,17.599048,0.550525,получено экспереметально,0.954381,получено экспереметально,0.981060,получено экспереметально
1,пшеница,овес,18.985714,0.712560,получено экспереметально,0.990260,"получено из пшеница, ячмень, донник(вика)клеве...",1.000000,получено экспереметально
2,пшеница,ячмень,20.750000,0.712560,получено экспереметально,0.977442,получено экспереметально,1.000000,получено экспереметально
3,пшеница,пар,0.000000,NaN,получено экспереметально,NaN,получено экспереметально,NaN,получено экспереметально
4,пшеница,оз. рожь,27.769808,0.783050,"получено из пшеница, овес, ячмень",0.990260,"получено из пшеница, ячмень, донник(вика)клеве...",1.000000,получено экспереметально
5,пшеница,донник(вика)клевер,38.244246,0.682186,"получено из пшеница, овес, ячмень",1.000000,получено экспереметально,0.997633,"получено из пшеница, овес, ячмень, оз. рожь, в..."
6,пшеница,вика-овес,23.592412,0.682186,"получено из пшеница, овес, ячмень",1.000000,получено экспереметально,1.000000,получено экспереметально
7,пшеница,рапс,5.908361,0.682186,"получено из пшеница, овес, ячмень",1.000000,получено экспереметально,1.000000,получено экспереметально
8,пшеница,клевер,39.717313,0.682186,"получено из пшеница, овес, ячмень",1.000000,получено экспереметально,1.000000,получено экспереметально
9,пшеница,вика-овес (з/м),26.973991,0.682186,"получено из пшеница, овес, ячмень",1.000000,получено экспереметально,1.000000,получено экспереметально


In [22]:
tensor.to_excel('tensor.xlsx')